In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc,from_json,col
import pyspark.sql.functions as psf
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pyspark --version


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.4.1
      /_/
                        
Using Scala version 2.12.17, OpenJDK 64-Bit Server VM, 1.8.0_92
Branch HEAD
Compiled by user centos on 2023-06-19T23:01:01Z
Revision 6b1ff22dde1ead51cbf370be6e48a802daae58b6
Url https://github.com/apache/spark
Type --help for more information.


In [3]:
import os
spark_version = '3.4.1'
scala_version = '2.12'
# TODO: Ensure match above values match the correct versions
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.5.1'
]

In [4]:
spark=SparkSession\
.builder\
.master("local[1]")\
.appName("Spark API")\
.config("spark.jars.packages", ",".join(packages))\
.config("spark.jars", "/home/kafka2/documents/orderManagement/postgresql-42.6.0.jar") \
.getOrCreate()



:: loading settings :: url = jar:file:/home/kafka2/miniconda3/envs/myenv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kafka2/.ivy2/cache
The jars for the packages stored in: /home/kafka2/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-795e9819-2c84-40ba-aea7-1589ab3fe158;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.1 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.5.1 in central
	found com.github.luben#zstd-jni;1.5.5-1 in centra

In [5]:
spark

## Consumer

- **movie_info** table

In [6]:
movie_df = spark \
.readStream \
.format("kafka") \
.option("kafka.bootstrap.servers", "192.168.1.14:9092") \
.option("subscribe", "fullfill27.test1.movie_info") \
.option("startingOffsets", "earliest") \
.load()\
.selectExpr("CAST(value AS STRING)")

In [7]:
temp1=movie_df.select(psf.get_json_object(movie_df['value'],"$.payload").alias('payload'))
temp1=temp1.select(psf.get_json_object(temp1['payload'],"$.before").alias('before'),\
                   psf.get_json_object(temp1['payload'],"$.after").alias('after'),\
                   psf.get_json_object(temp1['payload'],"$.op").alias('operation'),\
                   psf.get_json_object(temp1['payload'],"$.ts_ms").alias('timestamp debezium'),\
                  psf.get_json_object(temp1['payload'],"$.source").alias('source'))

temp_movie=temp1.select("before","after","operation",psf.get_json_object(temp1['source'],"$.ts_ms").alias('timestamp DB'),"timestamp debezium")


- **user_ratings** table

In [8]:
user_ratings_df = spark \
.readStream \
.format("kafka") \
.option("kafka.bootstrap.servers", "192.168.1.14:9092") \
.option("subscribe", "fullfill27.test1.user_ratings") \
.option("startingOffsets", "earliest") \
.load()\
.selectExpr("CAST(value AS STRING)")

In [9]:
temp1=user_ratings_df.select(psf.get_json_object(user_ratings_df['value'],"$.payload").alias('payload'))
temp1=temp1.select(psf.get_json_object(temp1['payload'],"$.before").alias('before'),\
                   psf.get_json_object(temp1['payload'],"$.after").alias('after'),\
                   psf.get_json_object(temp1['payload'],"$.op").alias('operation'),\
                   psf.get_json_object(temp1['payload'],"$.ts_ms").alias('timestamp debezium'),\
                  psf.get_json_object(temp1['payload'],"$.source").alias('source'))

temp_user_ratings=temp1.select("before","after","operation",psf.get_json_object(temp1['source'],"$.ts_ms").alias('timestamp DB'),"timestamp debezium")


## Spark Streaming Sink to PostgresSql

In [10]:
# Define a function to write data to postgres
def write_to_postgresql_movie(df,epoch_id):
    df.write \
    .format('jdbc') \
    .options(url='jdbc:postgresql://192.168.1.14/temp',
            driver='org.postgresql.Driver',
            dbtable='movie_info',
            user='postgres',
            password='Nintendo1',
            ) \
    .mode('append') \
    .save()
    
def write_to_postgresql_user_ratings(df,epoch_id):
    df.write \
    .format('jdbc') \
    .options(url='jdbc:postgresql://192.168.1.14/temp',
            driver='org.postgresql.Driver',
            dbtable='user_ratings',
            user='postgres',
            password='Nintendo1',
            ) \
    .mode('append') \
    .save()

-  **foreachBatch** in **PySpark** 

In [ ]:
query1=temp_movie.writeStream \
    .trigger(processingTime='5 seconds') \
    .outputMode('Append') \
    .foreachBatch(write_to_postgresql_movie) \
    .start()

query2=temp_user_ratings.writeStream \
    .trigger(processingTime='5 seconds') \
    .outputMode('Append') \
    .foreachBatch(write_to_postgresql_user_ratings) \
    .start()
spark.streams.awaitAnyTermination()


24/02/26 15:39:24 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9701da6b-a17b-4551-8637-8f00431a3e75. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/02/26 15:39:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/02/26 15:39:25 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-caa2aa9b-bdd9-4b59-a11b-de723bdebbc5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/02/26 15:39:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not support